# [In Progress]

# Programación Entera: Heurísticas para el Problema de Localización de Servicios

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Introducción](#Introducción)
  * [Problemas de Localización de Servicios](#Problemas-de-Localización-de-Servicios)
    * [Cubrimiento Total](#Cubrimiento-Total)
  * [Conclusiones](#Conclusiones)

## Introducción

TODO

#### Configuración del Entorno

In [1]:
rm(list = ls())

In [2]:
library(slam, warn.conflicts = FALSE)
library(Rglpk, warn.conflicts = FALSE)
library(purrr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(ggthemes, warn.conflicts = FALSE)
library(scales, warn.conflicts = FALSE)

Using the GLPK callable library version 4.65


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 600)

#### Constantes y Funciones de Apoyo

In [4]:
Solve <- function(...) {
    Rglpk_solve_LP(...)
}

## Problemas de Localización de Servicios

TODO

### Cubrimiento Total

#### Descripción

TODO

#### Modelo

TODO

\begin{equation}
    \begin{array}{ll@{}ll}
      \text{Minimizar} & \displaystyle \sum\limits_{j=1}^{n} f_{j} \cdot x_{j} +  \sum\limits_{i=1}^{m}\sum\limits_{j=1}^{n} c_{ij} \cdot z_{ij} \\
      \text{sujeto a}  & \sum\limits_{j=1}^{n} z_{ij} \geq d_i, & \forall i \in \{1,...,m\}\\
                       & \sum\limits_{i=1}^{m} z_{ij} - s_j \cdot x_{j} \leq 0, & \forall j \in \{1,...,n\}\\
                       & z_{ij} \geq 0, & \forall i \in \{1,...,m\}, \forall j \in \{1,...,n\} \\
                       & x_{j} \in \{0, 1\}, & \forall j \in \{1,...,n\}
    \end{array}
\end{equation}

#### Ejemplo: WebBook

Web Book vende libros por internet. La administración quiere saber en qué sitios se deben ubicar almacenes de distribución, de entre $5$ sitios posibles, que denotaremos por *A1*, *A2*, *A3*, *A4* y *A5*. La región de ventas está dividida en $5$ zonas indicadas por *NO*, *SO*, *MO*, *SE* y *NE*. Se proporcionan los siguientes datos: la demanda semanal promedio de cada región (en número de libros), el costo promedio de envío de un libro (Euros/libro), el costo fijo semanal (en Euros) si el almacén está en funcionamiento, y la capacidad de cada almacén (en número de libros).

In [5]:
(p <- list(n = 5,
     m = 5,
     c = matrix(c(2.40, 3.50, 4.80, 6.80, 5.75,
                  3.25, 2.30, 3.40, 5.25, 6.00,
                  4.05, 3.25, 2.85, 4.30, 4.75,
                  5.25, 6.05, 4.30, 3.25, 2.75,
                  6.95, 5.85, 4.80, 2.10, 3.50), 
                nrow = 5, byrow = TRUE),
     f = c(4000, 3000, 2500, 4000, 3000),
     s = c(20000, 20000, 15000, 25000, 15000),
     d = c(8000, 12000, 9000, 14000, 17000)
))

2.40,3.50,4.80,6.80,5.75
3.25,2.30,3.40,5.25,6.00
4.05,3.25,2.85,4.30,4.75
5.25,6.05,4.30,3.25,2.75
6.95,5.85,4.80,2.10,3.50


Un directivo de la compañía piensa que la mejor opción es abrir los almacenes *A1*, *A2* y *A4*. Para esta opción, calcular manualmente su correspodiente costo.

In [6]:
filtered <- c(1, 2, 4)

SolveFiltered <- function(p, filtered = NULL) {

    conditions <- matrix(0, nrow = p$n + p$m, ncol = (1 + p$m) * p$n)

    for (i in 1:p$m) {
        conditions[i, seq(p$n * (i - 1) + 1, length.out = p$n)] <- 1
    }

    for (i in 1:p$n) {
        conditions[p$m + i, seq(i, length.out = p$m, by = p$n)] <- 1
        if (is.null(filtered) || i %in% filtered) {
            conditions[p$m + i, p$m * p$n + i] <- -p$s[i]   
        }
    }

    rhs <- c(p$d, rep(0, p$n))
    obj <- c(p$c, p$f)

    dir <- rep(c(">=", "<="), c(p$m, p$n))
    types <- rep(c("C", "B"), c(p$m, 1) * p$n)
    
    s <- Solve(obj = obj, 
               mat = conditions, 
               dir = dir, 
               rhs = rhs,
               types = types)
    
    return(s)
}

s <- SolveFiltered(p, filtered)
s$optimum
matrix(s$solution[1:(p$m * p$n)], nrow = p$n)
matrix(s$solution[(p$m * p$n + 1):(p$n * (1 + p$m))], nrow = 1)




[1] 199850

8000,1000,0,0,6000
0,11000,9000,0,0
0,0,0,0,0
0,0,0,14000,11000
0,0,0,0,0


1,1,0,1,0


Añadir las siguientes restricciones:
  * Por proximidad geográfica, los almacenes *A1* y *A3* no deberían estar abiertos simultáneamente.
  * Para mantener un equilibrio entre los distribuidores, si al almacén *A4* se le asignan más de $22.000$ libros semanales, los almacenes *A1* y *A2* deben tener asignados, en conjunto, al menos $15.000$ libros.

In [7]:
conditions <- matrix(0, nrow = p$n + p$m, ncol = (1 + p$m) * p$n)

for (i in 1:p$m) {
    conditions[i, seq(p$n * (i - 1) + 1, length.out = p$n)] <- 1
}

for (i in 1:p$n) {
    conditions[p$m + i, seq(i, length.out = p$m, by = p$n)] <- 1
    conditions[p$m + i, p$m * p$n + i] <- -p$s[i]
}

rhs <- c(p$d, rep(0, p$n))
obj <- c(p$c, p$f)

dir <- rep(c(">=", "<="), c(p$m, p$n))
types <- rep(c("C", "B"), c(p$m, 1) * p$n)

s <- Solve(obj = obj, 
          mat = conditions, 
          dir = dir, 
          rhs = rhs,
          types = types)

s$optimum
matrix(s$solution[1:(p$m * p$n)], nrow = p$n)
matrix(s$solution[(p$m * p$n + 1):(p$n * (1 + p$m))], nrow = 1)

[1] 165100

8000,0,0,0,0
0,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


1,1,1,1,1


In [8]:
a1 <- rep(0, (1 + p$m) * p$n)
a1[p$m * p$n + c(1, 3)] <- 1
conditions <- rbind(conditions, a1)
dir <- c(dir, "<=")
rhs <- c(rhs, 1)

s <- Solve(obj = obj, 
          mat = conditions, 
          dir = dir, 
          rhs = rhs,
          types = types)

s$optimum
matrix(s$solution[1:(p$m * p$n)], nrow = p$m)
matrix(s$solution[(p$m * p$n + 1):(p$m * (1 + p$n))], nrow = 1)

[1] 167900

0,0,0,0,0
8000,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


0,1,1,1,1


In [9]:
obj <- c(obj, 0)
conditions <- cbind(conditions, rep(0, nrow(conditions)))
types <- c(types, "B")

a2 <- conditions[4, ]
a2[length(a2)] <- -p$s[4]
conditions <- rbind(conditions, a2)
dir <- c(dir, "<=")
rhs <- c(rhs, 22000)

a3 <- colSums(conditions[c(1, 2), ])
conditions <- rbind(conditions, a3)
dir <- c(dir, ">=")
rhs <- c(rhs, 15000)

s <- Solve(obj = obj, 
          mat = conditions, 
          dir = dir, 
          rhs = rhs,
          types = types)

s$optimum
matrix(s$solution[1:(p$m * p$n)], nrow = p$m)
matrix(s$solution[(p$m * p$n + 1):(p$m * (1 + p$n))], nrow = 1)

[1] 167900

0,0,0,0,0
8000,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


0,1,1,1,1


¿Cómo se debe modificar el modelo de distribuci ́on si se exige que cada región de ventas debe ser asignada a un único almacén?

In [10]:
obj.1 <- c(obj, rep(0, p$m * p$n))
types.1 <- c(types, rep("B", p$m *p$n))
conditions.1 <- cbind(conditions, matrix(0, ncol = p$m * p$n, nrow = nrow(conditions)))


id1 <- ncol(conditions)
integer.res <- matrix(0, ncol = ncol(conditions.1), nrow = p$m * p$n + p$n)


for (i in 1:p$m) {
    for (j in 1:p$n) {
        integer.res[j + (i - 1) * p$n, j + (i - 1) * p$n] <- 1
        integer.res[j + (i - 1) * p$n, id1 +  j + (i-1) * p$n] <- -min(p$d[i], p$s[j])
    }
}

for (i in 1:p$n) {
    integer.res[p$n * p$m + i, seq(i + id1, length.out = p$m, by = p$n)] <- 1
}


conditions.1 <- rbind(conditions.1, integer.res)
dir.1 <- c(dir, rep("<=", p$m * p$n + p$n))
rhs.1 <- c(rhs, rep(c(0, 1), c(p$m*p$n, p$n)))
      
s <- Solve(obj = obj.1, 
          mat = conditions.1, 
          dir = dir.1, 
          rhs = rhs.1,
          types = types.1)

s$optimum
matrix(s$solution[1:(p$m * p$n)], nrow = p$m)
matrix(s$solution[(p$m * p$n + 1):(p$m * (1 + p$n))], nrow = 1)

[1] 0

0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0


0,0,0,0,0


## Conclusiones

TODO

## Referencias

  * TODO